# 実行内容
- s3からBIG Eの平均価格を取得
- 画面の値と比較

In [12]:
import sys

sys.path.append("../")
from datetime import datetime

import pandas as pd
import pymysql
from sshtunnel import SSHTunnelForwarder

from vook_db_v7.local_config import get_ec2_config, get_rds_config
from vook_db_v7.rds_handler import get_products, read_sql_file

In [13]:
# get_products()

In [14]:
config_ec2 = get_ec2_config()
query = read_sql_file("../vook_db_v7/sql/products.sql")
df_from_db = pd.DataFrame()
with SSHTunnelForwarder(
    (config_ec2["host_name"], config_ec2["ec2_port"]),
    ssh_username=config_ec2["ssh_username"],
    ssh_pkey=config_ec2["ssh_pkey"],
    remote_bind_address=(
        config_ec2["rds_end_point"],
        config_ec2["rds_port"],
    ),
) as server:
    print(f"Local bind port: {server.local_bind_port}")
    conn = None
    try:
        conn = pymysql.connect(
            **get_rds_config(server.local_bind_port),
            connect_timeout=10,
        )
        cursor = conn.cursor()
        cursor.execute(query)
        for row in cursor:  # column1, column2, ...は取得したいカラム名に合わせて変更してください
            df_from_db = pd.concat([df_from_db, pd.DataFrame([row])], ignore_index=True)
    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if conn is not None:
            conn.close()

Local bind port: 63360


In [15]:
df_from_db.shape

(4174, 9)

In [16]:
df_from_db

,id,name,url,price,knowledge_id,platform_id,size_id,created_at,updated_at
0,16828,【中古】未使用品 リーバイス Levi's 501 66前期 70's ヴィンテージ デッド...,https://hb.afl.rakuten.co.jp/hgc/g00qosae.brn1...,601000,5,1,999,2024-01-27 13:45:40.711658,2024-01-27 13:45:40.711658
1,16829,【中古】未使用品 リーバイス Levi's 501 66前期 70's ヴィンテージ デッド...,https://hb.afl.rakuten.co.jp/hgc/g00rsgoe.brn1...,601000,5,1,999,2024-01-27 13:45:40.711658,2024-01-27 13:45:40.711658
2,16830,Levi's(リーバイス) サイズ:W31/31 70S VINTAGE 501 66前期 ...,https://hb.afl.rakuten.co.jp/hgc/g00s6rie.brn1...,423500,5,1,999,2024-01-27 13:45:40.711658,2024-01-27 13:45:40.711658
3,16831,Levi's(リーバイス) サイズ:W29/30 70S VINTAGE 501 66前期 ...,https://hb.afl.rakuten.co.jp/hgc/g00s6rie.brn1...,423500,5,1,999,2024-01-27 13:45:40.711658,2024-01-27 13:45:40.711658
4,16832,Levi's(リーバイス) サイズ:31×36 70S VINTAGE 501 66前期 デ...,https://hb.afl.rakuten.co.jp/hgc/g00s6rie.brn1...,363000,5,1,999,2024-01-27 13:45:40.711658,2024-01-27 13:45:40.711658
...,...,...,...,...,...,...,...,...,...
4169,21409,【中古】Lee◆Gジャン/34/デニム/IDG/101-J【レディースウェア】,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,5390,107,1,999,2024-01-27 13:50:05.859605,2024-01-27 13:50:05.859605
4170,21410,リー 220-J 80s USA製 デニム ジャケット 10MED Lee 101-J 黒タ...,https://hb.afl.rakuten.co.jp/hgc/g00rf2ie.brn1...,4944,107,1,999,2024-01-27 13:50:05.859605,2024-01-27 13:50:05.859605
4171,21411,リー 220-J 80s USA製 デニム ジャケット 10MED Lee 101-J 黒タ...,https://hb.afl.rakuten.co.jp/hgc/g00u3aie.brn1...,4944,107,1,999,2024-01-27 13:50:05.859605,2024-01-27 13:50:05.859605
4172,21412,50's LEVI'S 503BXX 革パッチ最終 両面タブ デニムパンツ W28 50年代...,https://hb.afl.rakuten.co.jp/hgc/g00taeee.brn1...,217800,109,1,999,2024-01-27 13:50:09.100133,2024-01-27 13:50:09.100133


In [38]:
# 本番反映前
# 現在の平均価格 BIG E テスト
# 平均価格
# 63,361円
# 最低価格
# 7,590円
# 最高価格
# 536,250円

In [17]:
# 現時点
(
    df_from_db[df_from_db["knowledge_id"] == 10]["price"].mean(),
    df_from_db[df_from_db["knowledge_id"] == 10]["price"].min(),
    df_from_db[df_from_db["knowledge_id"] == 10]["price"].max(),
)

(65158.25102880658, 2530, 673200)

In [18]:
# 現時点
# 平均価格
# 65,158円
# 最低価格
# 2,530円
# 最高価格
# 673,200円